### Code reference from (代码参考) https://github.com/krasserm/super-resolution

# Import

In [1]:
import shutil
shutil.rmtree('.div2k/caches')    #递归删除文件夹

from data import DIV2K
train = DIV2K(scale=2, downgrade='bicubic', subset='train')
train_ds = train.dataset(batch_size=16, random_transform=True)

Caching decoded images in .div2k/caches\DIV2K_train_LR_bicubic_X2.cache ...
Cached decoded images in .div2k/caches\DIV2K_train_LR_bicubic_X2.cache.
Caching decoded images in .div2k/caches\DIV2K_train_HR.cache ...
Cached decoded images in .div2k/caches\DIV2K_train_HR.cache.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [1]:
import os
import time
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image

from tensorflow.keras.layers import Add, Conv2D, Input, Lambda, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
#policy = mixed_precision.Policy('float32')
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  GeForce GTX 1050 Ti, compute capability 6.1
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Compute dtype: float16
Variable dtype: float32


# EDSR

In [4]:
DIV2K_RGB_MEAN = np.array([0.4488, 0.4371, 0.4040]) * 255
def pixel_shuffle(scale):
    return lambda x: tf.nn.depth_to_space(x, scale)
def normalize(x):
    return (x - DIV2K_RGB_MEAN) / 127.5
def denormalize(x):
    return x * 127.5 + DIV2K_RGB_MEAN
def upsample(x, scale, num_filters):
    def upsample_1(x, factor, **kwargs):
        """Sub-pixel convolution."""
        x = Conv2D(num_filters * (factor ** 2), 3, padding='same', **kwargs)(x)
        return Lambda(pixel_shuffle(scale=factor))(x)
    if scale == 2:
        x = upsample_1(x, 2, name='conv2d_1_scale_2')
    elif scale == 3:
        x = upsample_1(x, 3, name='conv2d_1_scale_3')
    elif scale == 4:
        x = upsample_1(x, 2, name='conv2d_1_scale_2')
        x = upsample_1(x, 2, name='conv2d_2_scale_2')
    return x
def res_block(x_in, filters):
    """Creates an EDSR residual block."""
    x = Conv2D(filters, 3, padding='same', activation='relu')(x_in)
    x = Conv2D(filters, 3, padding='same')(x)
    x = Add()([x_in, x])
    return x
def edsr(scale, num_filters=25, num_res_blocks=4):
    """Creates an EDSR model."""
    #input normalize
    x_in = Input(shape=(None, None, 3))
    x = Lambda(normalize)(x_in)
    
    #res blocks and add
    x = Conv2D(num_filters, 3, padding='same')(x)
    for i in range(num_res_blocks):
        x = res_block(x, num_filters)
    x = Conv2D(num_filters, 3, padding='same')(x)
    
    #up sample
    x = upsample(x, scale, num_filters)
    x = Conv2D(3, 3, padding='same')(x)
    
    #output denormalize
    x = Lambda(denormalize)(x)
    return Model(x_in, x, name="edsr")

# Train

In [11]:
def PSNR(img_hr, img_sr):
    return 10.0*K.log(K.square(255.0)/(K.mean(K.square(img_sr-img_hr))))/K.log(10.0)

# Create directory for saving model weights
weights_dir = 'weights/article'
# EDSR baseline as described in the EDSR paper (1.52M parameters)
model_edsr = edsr(scale=2, num_filters=25, num_res_blocks=4)

learning_rate = 1e-4
epochs = 30
steps_per_epoch = 1000

os.makedirs(weights_dir, exist_ok=True)

#####################################################################################################
model_edsr.load_weights(os.path.join(weights_dir, 'weights-edsr-x2.h5'))
model_edsr.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='mean_absolute_error',
    metrics=[PSNR, "accuracy"])
model_edsr.summary()

filepath=os.path.join(weights_dir, 'weights-edsr-x2-{epoch:04d}-{PSNR:.2f}.h5')
checkpoint = ModelCheckpoint(filepath, monitor=PSNR, verbose=1, mode='max')
callbacks_list = [checkpoint]

model_edsr.fit(
    train_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    callbacks=callbacks_list,
    verbose=1)
model_edsr.save_weights(os.path.join(weights_dir, 'weights-edsr-x2.h5'))



Model: "edsr"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
lambda_12 (Lambda)              (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, None, None, 2 700         lambda_12[0][0]                  
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, None, None, 2 5650        conv2d_44[0][0]                  
_______________________________________________________________________________________________

# DEMO

In [5]:
def resolve_single(model, lr):
    return resolve(model, tf.expand_dims(lr, axis=0))[0]

def resolve(model, lr_batch):
    lr_batch = tf.cast(lr_batch, tf.float32)
    sr_batch = model(lr_batch)
    sr_batch = tf.clip_by_value(sr_batch, 0, 255)
    sr_batch = tf.round(sr_batch)
    sr_batch = tf.cast(sr_batch, tf.uint8)
    return sr_batch

#def load_image(path):
#    return np.array(Image.open(path))

# DEMO Picture

In [8]:
edsr_pre_trained = edsr(scale=2, num_filters=25, num_res_blocks=4)
edsr_pre_trained.load_weights('weights/article/weights-edsr-x2.h5')
#edsr_pre_trained.load_weights('weights/article/weights-edsr-x2.h5')

#####################################################
tick0=time.time()

for i in range(10):
    #read image
    lr_image_path = './demo/testys1080p001.png'
    lr = np.array(Image.open(lr_image_path))
    #run
    sr = resolve_single(edsr_pre_trained, lr)
    #write image
    sr_image_path = './testys1080p001_out.bmp'
    data_numpy_sr = Image.fromarray(sr.numpy())
    data_numpy_sr.save(sr_image_path)

tick1=time.time()
print(tick1-tick0)

#####################################################
tick0=time.time()
#read image
lr_image_path = './demo/testys1080p001.bmp'
lr = np.array(Image.open(lr_image_path))

tick1=time.time()
#run
sr = resolve_single(edsr_pre_trained, lr)
tick2=time.time()
#write image
sr_image_path = './testys1080p001_out.bmp'
data_numpy_sr = Image.fromarray(sr.numpy())
data_numpy_sr.save(sr_image_path)
tick3=time.time()

print(tick1-tick0)
print(tick2-tick1)
print(tick3-tick2)

7.393781423568726
0.015028953552246094
0.48154687881469727
0.04499197006225586


# DEMO Video

In [6]:
edsr_pre_trained = edsr(scale=2, num_filters=25, num_res_blocks=4)
edsr_pre_trained.load_weights('weights/article/weights-edsr-x2.h5')

cap = cv2.VideoCapture("demo/test0_l.flv")  #读取视频文件
fps = 30
size = (int(2*cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(2*cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
videoWriter = cv2.VideoWriter('test0_l.avi', cv2.VideoWriter_fourcc('X', 'V', 'I', 'D'), fps, size)
#PS: I后缀一般指代帧内编码
#cv2.VideoWriter_fourcc('I', '4', '2', '0'),该参数是YUV编码类型，文件名后缀为.avi
#cv2.VideoWriter_fourcc('P', 'I', 'M', 'I'),该参数是MPEG-1编码类型，文件名后缀为.avi
#cv2.VideoWriter_fourcc('M', 'P', 'E', 'G'),该参数是MPEG-2编码类型，文件名后缀为.avi
#cv2.VideoWriter_fourcc('X', 'V', 'I', 'D'),该参数是MPEG-4编码类型，文件名后缀为.avi
#cv2.VideoWriter_fourcc('T', 'H', 'E', 'O'),该参数是Ogg Vorbis,文件名后缀为.ogv
#cv2.VideoWriter_fourcc('F', 'L', 'V', '1'),该参数是Flash视频，文件名后缀为.flv
#cv2.VideoWriter_fourcc('M','J','P','G') = motion-jpeg codec   --> mp4v
#cv2.VideoWriter_fourcc('M', 'P', '4', '2') = MPEG-4.2 codec
#cv2.VideoWriter_fourcc('D', 'I', 'V', '3') = MPEG-4.3 codec
#cv2.VideoWriter_fourcc('D', 'I', 'V', 'X') = MPEG-4 codec     --> avi
#cv2.VideoWriter_fourcc('U', '2', '6', '3') = H263 codec
#cv2.VideoWriter_fourcc('I', '2', '6', '3') = H263I codec

counter = 0
showTimeLine = 1
tick0 = time.time()
while(True):
    ret, lr = cap.read()
    if ret:
        sr = resolve_single(edsr_pre_trained, lr)#process from lr to sr
        videoWriter.write(sr.numpy())#write sr
        counter = counter+1#counter for compute fps
        if showTimeLine == 1:#show TimeLine
            lr = cv2.resize(lr, (960, 540))
            cv2.imshow("TimeLine", lr)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
tick1= time.time()
print('time:', tick1-tick0)
print('fps:', counter/(tick1-tick0))

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


time: 139.15846848487854
fps: 1.6599780273171179
